In [1]:
from transformers import BertConfig, BertTokenizer, BertModel
import torch
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import re

/Users/colevincentfrank/Documents/Documents_Macbook_Air_2021_cvf/Coursework/MSCAPP/2022_Fall/deep_learning_systems/lottery-ticket/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src import analyze_feedforward_mnist

/Users/colevincentfrank/Documents/Documents_Macbook_Air_2021_cvf/Coursework/MSCAPP/2022_Fall/deep_learning_systems/lottery-ticket


ModuleNotFoundError: No module named 'lt_model'

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### BERT-base Overview
[bert-base-uncased](https://huggingface.co/bert-base-uncased) has 110M parameters and was pretrained on BookCorpus and English Wikipedia on two tasks: masked language modeling (MLM) and next sentence predicition (NSP).
<br>
It's architecture is a multi-layer bidirectional Transformer with 12 layers, a hidden dimension of 768, and 12 attention heads. Each of the 12 layers is of the form:

#### Attention
|        Self-Attention (in=768, out=768)
    <br>
|        Dropout
    <br>
|        Self-Attention Output Feed-Forward (in=768, out=768)
    <br>
|        Layer Norm
    <br>
|        Dropout

#### Intermediate

|       Feed-Forward (in=768, out=3072)
    <br>
|       GELU Activation

#### Output

|       Feed-Forward (in=3072, out=768)
    <br>
|       Layer Norm
    <br>
|        Dropout
<br>
There is also a final "BertPooler" layer that consists of a Feed Forward (in=768, out=768) and a Tanh() activation
    


In [24]:
model.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)